In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import selection_tests

In [2]:
#need to replicate table 3?
school_data = pd.read_stata('aerdata_colombia2.dta')
#keep if checkid==1 & age>=9 & age<=25
school_data = school_data[(school_data['checkid']==1) &
                          (school_data['age']>=9) & (school_data['age']<=25) ]
print(school_data.describe())

            vouch0     sex_name      match_i     match_ic     match_i7  \
count  3542.000000  3542.000000  3542.000000  3542.000000  3542.000000   
mean      0.585263     0.494918     0.354037     0.338509     0.331451   
std       0.492746     0.500045     0.478288     0.473269     0.470801   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       1.000000     0.000000     0.000000     0.000000     0.000000   
75%       1.000000     1.000000     1.000000     1.000000     1.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

               age        phone  checkid         math         read  \
count  3542.000000  3542.000000   3542.0  3541.000000  3541.000000   
mean     12.670243     0.886222      1.0    14.713037    16.355835   
std       1.295708     0.317586      0.0    20.436213    22.758410   
min      10.000000     0.000000      1.0     0.000000

In [3]:
# Fit the model with ols = True
x_names =['vouch0', 'age', 'sex_name'] 
y_name1 = 'read'
y_name2 = 'math'

#setup x/y for missing data models
x = school_data[x_names].copy()
y1 = school_data[y_name1]
y2 = school_data[y_name2]

print(stats.mode(school_data[y_name1])[0][0])
print(stats.mode(school_data[y_name2])[0][0])

x = sm.add_constant(x[~np.isnan(y1)])
y1 = y1[~np.isnan(y1)]
y2 = y2[~np.isnan(y2)]

0.0
0.0


In [4]:
cc_model1 = selection_tests.Tobit(y1,x, cc=True).fit()
cc_model2 = selection_tests.Tobit(y2,x, cc=True).fit()
#why is there randomly a missing observation?
print(cc_model1.summary(title='OLS CC'))
print(cc_model2.summary(title='OLS CC'))

Optimization terminated successfully.
         Current function value: 1.356813
         Iterations: 114
         Function evaluations: 189
Optimization terminated successfully.
         Current function value: 1.315860
         Iterations: 102
         Function evaluations: 172
                                    OLS CC                                    
Dep. Variable:                   read   Log-Likelihood:                -4804.5
Model:                          Tobit   AIC:                             9617.
Method:            Maximum Likelihood   BIC:                             9642.
Date:                Fri, 30 Jul 2021                                         
Time:                        16:23:16                                         
No. Observations:                3541                                         
Df Residuals:                    3537                                         
Df Model:                           3                                         
         

In [5]:
# Fit the model with ols = False
tobit_model1 = selection_tests.Tobit(y1,x).fit()
tobit_model2 = selection_tests.Tobit(y2,x).fit()
print(tobit_model1.summary())
print(tobit_model2.summary())

/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                Tobit Results                                 
Dep. Variable:                   read   Log-Likelihood:                -7519.5
Model:                          Tobit   AIC:                         1.505e+04
Method:            Maximum Likelihood   BIC:                         1.507e+04
Date:                Fri, 30 Jul 2021                                         
Time:                        16:23:18                                         
No. Observations:                3541                                         
Df Residuals:                    3537                                         
Df Model:                           3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        304.9482     12.413     24.568      0.000     280.620     329.276
vouch0         8.8082      1.995      4.415      0.0

/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [7]:
print(selection_tests.test_table(y1,x,selection_tests.setup_test))
print(selection_tests.test_table(y2,x,selection_tests.setup_test))

/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


V ----
-0.39724847183870265
----
---- bootstrap: llr, omega ----
2714.592065799726 72.2408255001202
----


/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


regular: test, llr, omega ----
37.58247909649161 2714.9893142715646 1.214002934895893
---- 


/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


...

/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


V ----
-0.4667908112720395
----
---- bootstrap: llr, omega ----
2742.7549718059604 73.13805400173513
----


/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


regular: test, llr, omega ----
37.507448072820644 2743.2217626172323 1.2290808084762685
---- 
37.50106574808363 2.067102365167206
37.507448072820644
\begin{center}
\begin{tabular}{ccc}
\toprule
\textbf{Version} & \textbf{Result} & \textbf{95 \% CI} \\ \midrule
Bootstrap & H1 & [35.906, 39.034] \\
Shi (2015) & H1 & [-1.067, 3.067] \\
Classical & H1 & [35.548, 39.466] \\
\bottomrule
\end{tabular}
\end{center}
None


/home/erichschulman/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
